## Imports

In [ ]:
import lfd
import pandas as pd
import numpy as np
import shutil
lfd.set_logging(stdout_level='INFO')
pd.set_option('display.max_columns', None)  # Makes Pandas show all columns with showing tablesa
pd.set_option('display.max_rows', 70)

In [ ]:
%reload_ext autoreload
%autoreload 2

In [ ]:
DATASET = 'Mnist'
TARGET = 'label'
SET_ASIDE = [TARGET]

In [ ]:
data = lfd.Visual().load(f'../../learnfromdata_new/datasets/visual/{DATASET.lower()}', load_images=True)

In [ ]:
counts = data.df[TARGET].value_counts().to_dict()
new_count = int(np.median(list(counts.values())))
balance_params = dict(target=TARGET, seed=0, stratified=None, counts={k: new_count for k in counts.keys()})

params = dict(
    set_aside = SET_ASIDE,
    data = dict(
        add_noise = None,#dict(seed=0),
        test_split = dict(test_size=0.2, stratify_col=None, seed=0),
        train_balance = balance_params,
        #valid_split = dict(test_size=0.2, stratify_col=None, seed=0),
    ),
    transform = dict(
        # uniselector = dict(min_occ=0.01, max_occ=0.99),
        # encoder = dict(min_occ=0.01, method='onehot', target=TARGET),
        # biselector = dict(threshold=0.8, target=TARGET),
    ),
    model = dict(
        target=TARGET, mode='multiclass', seed_train=0,
        base0 = dict(algorithm='neuralnet', name='NeuralNet', 
            hyper_params=dict(
                transfer_learning=None,#'https://tfhub.dev/google/imagenet/inception_v3/classification/4', 
                #transfer_learning='https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet1k_s/classification/2', 
                epochs=10, activation='tanh', batch_size=16, learning_rate=0.01)),
        calibrate = None
    ),
)

In [ ]:
pipe = lfd.Pipeline(name=f'{DATASET}_{TARGET}').learn(
    params, data=data, evaluate=True, explain=False)

In [ ]:
pipe.save('../../experiments', slim=False, as_pickle=False)

In [ ]:
pipe.models['NeuralNet'].metrics

In [ ]:
pipe.models['NeuralNet'].confusion

In [ ]:
raise

In [ ]:
import folium
import pandas as pd

In [ ]:
import json
with open('countries.geojson') as f:
    coords = json.load(f)
countries = [x['properties']['ADMIN'] for x in coords['features']]

In [ ]:
timeseries = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
timeseries.dropna(subset=['Lat', 'Long'], inplace=True)
timeseries['location'] = timeseries[['Province/State', 'Country/Region']].bfill(axis=1).iloc[:, 0]
timeseries = timeseries[timeseries.location.isin(countries)]

In [ ]:
m = folium.Map(zoom_start = 10, tiles='cartodbdark_matter')
#folium.TileLayer('https://tiles.stadiamaps.com/tiles/alidade_smooth_dark/{z}/{x}/{y}{r}.png').add_to(my_map)
#folium.Marker(location=coords, popup='Location!',tooltip='Location', ).add_to(my_map)

# add marker one by one on the map
for i, row in timeseries.iterrows():
   folium.Circle(
      location=[row.Lat, row.Long],
      tooltip=row.location + ' ' + str(row['1/22/21']),
      radius=row['1/22/21']/20-1,
      color='crimson',
      fill=False,
      fill_color='crimson'
   ).add_to(m)
# Show the map again
m

In [ ]:
m = folium.Map(zoom_start = 10, tiles='cartodbdark_matter')

# generate choropleth map 
folium.Choropleth(
    name="choropleth",
    geo_data='countries.geojson',
    data=timeseries,
    columns=['location', '12/2/21'],
    key_on='properties.ADMIN',
    fill_color='YlGn', 
    fill_opacity=0.3,
    line_opacity=1,
    legend_name='Births per 1000 inhabitants',
    smooth_factor=0).add_to(m)

# display map
m
